# PC caluculation and ancestry assignment

AMP-PD data version 3 provided plink2 binary "all_chrs_merged" were processed with the following commands

    plink2\
     --mac 20
     --make-pgen
     --out all_chr_merged_mac20
     --pfile all_chrs_merged

In [ ]:
%%bash
mkdir -p temp
sh infer_pop.sh all_chr_merged_mac20 ../../1kg_p3/all_hg38_filtered_chrpos temp 2

In [ ]:
# %%bash
# # if wanted to do the analysis again. 
# cd temp
# INPUT=all_chr_merged_mac20
# python3 ../process_pca_results.py --input ${INPUT}_snp_ref_pca

# Calculate PRS

rs11578699 was substituted with rs11577197 (perfect LD) and rs3742785 was substituted by rs10134885 (R2=0.989)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
import argparse
import requests
import os
import time
import subprocess
import sys

In [ ]:
def submitTerminal(command, printing=False, message=''):
    # quick command to submit jobs to terminal
    start = time.time()
    res=subprocess.run(command.split(' '), stdout=subprocess.PIPE)
    end = time.time()
    sys.stdout.write('EXEC_TIME in sec: '+ str(round(end - start, 3)) + ' : ')
    if printing:
        print(res.stdout.decode('utf-8'))
    if message=='':
        return(res.stdout.decode('utf-8'))
    else:
        print(message, '\n')
        
def plink_cmd(plink_file_path, output_path, options):
    cmd = f'plink2 --pfile {plink_file_path} {options} --out {output_path} --threads 2'
    return cmd

In [ ]:
%%bash
mkdir -p prs
cd prs
wget https://raw.githubusercontent.com/neurogenetics/genetic-risk-score/master/META5_GRS_RSid.txt

In [ ]:
os.chdir('prs')

In [ ]:
t = pd.read_csv('META5_GRS_RSid.txt', delim_whitespace=True, header=None)

# We need to replace two SNPs to the proxy
t[1] = ['T' if rsid=='rs11578699'
        else 'C' if rsid=='rs3742785'
        else effect_allele for effect_allele, rsid in zip(t[1], t[0])]
t[0] = ['rs11577197' if rsid=='rs11578699' # Perfect LD in EUR
        else 'rs10134885' if rsid=='rs3742785' # Nearly perfect LD in EUR (R2=0.989)
        else rsid for rsid in t[0]]
t.to_csv('META5_GRS_RSid_proxy.txt', sep='\t', header=None, index=False)

In [ ]:
# create a plink2 file only contains meta5 proxies
score_path='META5_GRS_RSid_proxy.txt'
t= submitTerminal(plink_cmd('../all_chr_merged_mac20', 'meta5_proxy', 
                            f'--extract {score_path}  --make-pgen'), 
                            message='extract all meta5 variants')

In [ ]:
# Generate various PRS lists
df = pd.read_csv('META5_GRS_RSid_proxy.txt', delim_whitespace=True, names=['rsID', 'EffectAllele', 'Beta'])

snp_to_exclude1 = [
    'rs114138760', # GBA 
    'rs35749011',  # GBA, LD with E326K
    'rs76763715',  # GBA N370N
    ]  

snp_to_exclude2 = [
    'rs34637584',  # LRRK2 G2019S
    'rs76904798',  # LRRK2 
    ]  


snp_to_exclude3 = [
    'rs114138760', # GBA 
    'rs35749011',  # GBA, LD with E326K
    'rs76763715',  # GBA N370N
    'rs34637584',  # LRRK2 G2019S
    'rs76904798',  # LRRK2 
#     'rs5019538',   # SNCA
#     'rs13117519', # SNCA
    ]  

snp_to_exclude4 = [
    'rs11577197', # proxy
    'rs10134885', # proxy
    ] 

df1 = df[~df.rsID.isin(snp_to_exclude1)]
print(df1.shape)
df1.to_csv('META5_GRS_RSid_proxy_exclude_GBA.txt', sep='\t', header=None, index=False)

df2 = df[~df.rsID.isin(snp_to_exclude2)]
print(df2.shape)
df2.to_csv('META5_GRS_RSid_proxy_exclude_LRRK2.txt', sep='\t', header=None, index=False)

df3 = df[~df.rsID.isin(snp_to_exclude3)]
print(df3.shape)
df3.to_csv('META5_GRS_RSid_proxy_exclude_LRRK2_GBA.txt', sep='\t', header=None, index=False)


df4 = df3[~df3.rsID.isin(snp_to_exclude4)]
print(df4.shape)
df4.to_csv('META5_GRS_RSid_no_proxy_exclude_LRRK2_GBA.txt', sep='\t', header=None, index=False)

In [ ]:
score_path = 'META5_GRS_RSid.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'prs_no_proxy', f'--score {score_path}'), 
                  message='prs without proxy')

score_path = 'META5_GRS_RSid_proxy.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'prs_proxy', f'--score {score_path}'), 
                  message='prs with proxy')

score_path = 'META5_GRS_RSid_no_proxy_exclude_LRRK2_GBA.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'prs_no_proxy_exclude_LRRK2_GBA', f'--score {score_path}'), 
                  message='prs without proxy excluding 5 variants in LRRK2, GBA')

score_path = 'META5_GRS_RSid_proxy_exclude_LRRK2_GBA.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'prs_proxy_exclude_LRRK2_GBA', f'--score {score_path}'), 
                  message='prs with proxy excluding 5 variants in LRRK2, GBA')

score_path = 'META5_GRS_RSid_proxy_exclude_LRRK2.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'prs_proxy_exclude_LRRK2', f'--score {score_path}'), 
                  message='prs with proxy excluding 2 variants in LRRK2')

score_path = 'META5_GRS_RSid_proxy_exclude_GBA.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'prs_proxy_exclude_GBA', f'--score {score_path}'), 
                  message='prs with proxy excluding 3 variants in GBA')

In [ ]:
d1 = pd.read_csv('prs_no_proxy.sscore', delim_whitespace=True, index_col = 'IID', usecols=['IID', 'SCORE1_AVG'])
d2 = pd.read_csv('prs_no_proxy_exclude_LRRK2_GBA.sscore', delim_whitespace=True, index_col = 'IID', usecols=['IID', 'SCORE1_AVG'])
d3 = pd.read_csv('prs_proxy.sscore', delim_whitespace=True, index_col = 'IID', usecols=['IID', 'SCORE1_AVG'])
d4 = pd.read_csv('prs_proxy_exclude_LRRK2_GBA.sscore', delim_whitespace=True, index_col = 'IID', usecols=['IID', 'SCORE1_AVG'])
d5 = pd.read_csv('prs_proxy_exclude_GBA.sscore', delim_whitespace=True, index_col = 'IID', usecols=['IID', 'SCORE1_AVG'])
d6 = pd.read_csv('prs_proxy_exclude_LRRK2.sscore', delim_whitespace=True, index_col = 'IID', usecols=['IID', 'SCORE1_AVG'])
d = pd.concat([d1,d2,d3,d4,d5,d6], axis=1)
d.columns = ['PRS88','PRS83', 'PRSp90', 'PRSp85', 'PRSp87', 'PRSp88']
# PRS88 -> Full (missing 2 SNPs)
# PRS83 -> PRS88 Without GBA, LRRK2
# PRSp90 -> Used proxy above
# PRSp85 -> Used proxy above without GBA LRRK2
# PRSp87 -> Used proxy above without GBA
# PRSp88 -> Used proxy above without LRRK2
d_standardized= (d-d.mean())/d.std()
d_standardized.to_csv('prs.csv', index_label = 'IID')

# Create a raw file for the individual SNPs transposed

In [ ]:
#. Create transposed data
score_path = 'META5_GRS_RSid_proxy.txt'
t= submitTerminal(plink_cmd('meta5_proxy', 'meta5_proxy_transpose', f'--extract {score_path} --export A --export-allele {score_path}'),
                  message='dosage')